In [ ]:
# FIRST 表示增加此列为第一个列，AFTER表示增加在某个列之后
ALTER TABLE  bt_ws_list_2019q4 add COLUMN id serial8  comment 'id' after first_areaid

ALTER TABLE test ADD COLUMN c INT NOT NULL AFTER a

ALTER TABLE test ADD COLUMN id INT UNSIGNED NOT NULL auto_increment PRIMARY KEY FIRST

***********************************************************************************************************************************
# 经典topN问题
# 第一种
select *
from (
   select *, 
          row_number() over (partition by 姓名
                       order by 成绩 desc) as ranking
   from 成绩表) as a
where a.ranking <=2

# 第二种
select ifNull((select distinct salary from Employee order by Salary Desc limit 1,1),null) as SecondHighestSalary;

***********************************************************************************************************************************
# group by分组汇总后改变了表的行数，一行只有一个类别。而partiition by和rank函数不会减少原表中的行数
select *
from (select *, 
             avg(成绩) over (partition by 科目) as avg_score
      from 成绩表) as b
where 成绩 > avg_score;

***********************************************************************************************************************************
# 把时间戳转化为日期
select  from_unixtime(pay_time,'yyy-MM-dd hh:mm:ss')

# 把日期转化为时间戳
select UNIX_TIMESTAMP('2012-09-04 18:17:23')
***********************************************************************************************************************************
nulls last/nulls first
SELECT *,row_number()over(partition by first_areaid order by amount DESC NULLS LAST) as rank FROM table

***********************************************************************************************************************************
# mysql
select id,group_concat(name order by name desc Separator '-') from aa group by id;
 
# Hive
# Hive中实现group_concat功能（不用udf） 
# 注意:collect_set 只能返回不重复的集合 若要返回带重复的要用collect_list
SELECT id,concat_ws(',', collect_set(name)) FROM t GROUP BY id;

# postgresql
select deptno, string_agg(ename, ',' order by ename desc) from jinbo.employee group by deptno;
select array_agg(distinct deptno order by deptno desc) from jinbo.employee;

***********************************************************************************************************************************
# leader表

id       name

1        李一

2        刘二

3        张三

4        李四

5        王五


# staff表

id         leaderid

1          1,2,3

2           2

3          4,5

4          4

5          2,3,4


# 想要得到的结果

id         leaderid           领导姓名

1          1,2,3                 李一，刘二，张三

2           2                       刘二

3          4,5                    李四，王五

4          4                       李四



select staff.id,staff.leaderid,GROUP_CONCAT(leader.name) as '领导姓名'
from staff JOIN leader
ON FIND_IN_SET(leader.id,staff.leaderid)
GROUP BY staff.id

***********************************************************************************************************************************
# 查询爱好弹琴的所有人员
SELECT * from 
(SELECT p.id, p.`name`,GROUP_CONCAT(h.`name`) as hobby from  person p
LEFT JOIN hobby h ON FIND_IN_SET(h.id,p.hobby)
GROUP BY p.id) person1
WHERE FIND_IN_SET("弹琴",hobby)


SELECT hobby,GROUP_CONCAT(id order by id desc separator '_' ) from person1 GROUP BY hobby


# json key取值
select request_uri_dmalog_atype, get_json_object(request_uri_dmalog_show_post_id,'$.pn') from ods.ods_rtm_app_ev 

***********************************************************************************************************************************
SELECT ntile(3) OVER (ORDER BY SCORE DESC) as  rank,* from  Scores;

# MySQL 中 limit m,n 函数的含义是：从第 m+1 行开始读取 n 行；读取第101-200行，就是 limit 100,100

# 给出每个员工每年薪水涨幅超过5000的员工编号emp_no、薪水变更开始日期from_date以及薪水涨幅值salary_growth，并按照salary_growth逆序排列。
SELECT
  s2.emp_no,
  s2.from_date,
  (s2.salary - s1.salary) AS salary_growth
FROM
  salaries s1
INNER JOIN salaries s2 ON s1.emp_no = s2.emp_no
WHERE
  s2.salary - s1.salary > 5000
AND (
extract(YEAR FROM s2.to_date) - extract(YEAR FROM s1.to_date) = 1
)
ORDER BY
  salary_growth DESC;    

***********************************************************************************************************************************
# 强制使用指定索引    
select customer,count(1) c
from tem_table force index(idx_created)
where created between "2015-07-06" and "2015-07-07"
group by customer 
having c > 15
order by c desc     

***********************************************************************************************************************************
update tem_table set produce_second_addr = nullif(COALESCE(substring(prod_detail from '城市\":\"(.+?)\"'),''),'')::varchar(255)

UPDATE bt_taobao_product_china_all_detail_2018 a SET address1=b.first_addr||b.second_addr||b.third_addr
FROM bs_ec_business_base_all b WHERE a.factory_name=b.name ;

SELECT  COALESCE(month_amount_01,0)

ALTER TABLE bt_taobao_product_china_all_detail_2019
ADD address1 VARCHAR(255),
ADD area_brand1 VARCHAR(255),
ADD first_addr VARCHAR(50),
ADD second_add VARCHAR(50),
ADD third_addr VARCHAR(50),
ADD first_areaid VARCHAR(50),
ADD second_areaid VARCHAR(50),
ADD third_areaid VARCHAR(50);

#压缩表
CREATE TABLE bt_ws_list_2019q2_compress WITH (appendonly=true,compresstype=zlib,compresslevel=5) 
AS SELECT * FROM   bt_ws_list_2019q2 DISTRIBUTED BY (id)

ALTER TABLE  bt_ws_list_2019q2_compress  drop COLUMN id
ALTER TABLE  bt_ws_list_2019q2_compress  add COLUMN id serial8

SELECT (regexp_matches('买3送1 英国 袋泡茶20包*2g', '(\d+)[^\d+?][*](\d+)'))[1],
(regexp_matches('买3送1 英国 袋泡茶20包*2g', '(\d+)[^\d+?][*](\d+)'))[2]

select  regexp_matches('买3送1 英国 袋泡茶20包*2g', '\d+', 'g')
select  regexp_matches('买3送1 英国 袋泡茶20包*2g', '\d+')

select string_to_array('休闲,娱乐,运动,玩耍', ',');
SELECT array_to_string(regexp_matches('买3送1 英国 袋泡茶20包*2g', '\d+', 'g'),' ')

***********************************************************************************************************************************
# 更改表名 
alter table 表名 rename to 新表名

# 更改字段名
alter table 表名 rename 字段名 to 新字段名

# postgresql 修改字段类型
alter table  member alter  COLUMN  imgfileid  type int

# mysql 修改字段类型
alter table  member   modify  imgfileid  date

***********************************************************************************************************************************
# 去重 先增加自增id  再根据字段去重
ALTER table  pinduoduo_shop_unique add COLUMN id serial

delete from pinduoduo_shop_unique where id  not in (select max(id) from pinduoduo_shop_unique group by shop_id );

***********************************************************************************************************************************
# 动销率
select  
temp.prod_firstcat,
sum(temp.a),count(*),round(cast(sum(temp.a) as numeric)/cast(count(*) as numeric ),4)  as  "动销率"
from  
(select prod_firstcat, prod_id,(CASE when sum(COALESCE(month_amount_01,0)+ COALESCE(month_amount_02,0) + COALESCE(month_amount_03,0) +COALESCE(month_amount_04,0)+COALESCE(month_amount_05,0) +COALESCE(month_amount_06,0)+COALESCE(month_amount_07,0)+COALESCE(month_amount_08,0)+COALESCE(month_amount_09,0)+COALESCE(month_amount_10,0)+COALESCE(month_amount_11,0) +COALESCE(month_amount_12,0)) > 0 then 1 else 0   end )  as  a    
from   
gather_taobao.ir_taobao_product_trade_china_2018  group by prod_firstcat, prod_id ) as temp  
group by  temp.prod_firstcat

***********************************************************************************************************************************
数据库三范式
三范式
1 字段不能再分
2 不能部分依赖
3 不能传递依赖

***********************************************************************************************************************************


In [ ]:
SELECT CURRENT_DATE;
SELECT CURRENT_TIME;
SELECT CURRENT_TIMESTAMP;

# 日期截取函数
EXTRACT(日期元素 FROM 日期)

SELECT 
CURRENT_TIMESTAMP,
EXTRACT(YEAR FROM CURRENT_TIMESTAMP) AS year,
EXTRACT(MONTH FROM CURRENT_TIMESTAMP) AS month,
EXTRACT(DAY FROM CURRENT_TIMESTAMP) AS day,
EXTRACT(HOUR FROM CURRENT_TIMESTAMP) AS hour,
EXTRACT(MINUTE FROM CURRENT_TIMESTAMP) AS minute,
EXTRACT(SECOND FROM CURRENT_TIMESTAMP) AS second;

***********************************************************************************************************************************
# EXISTS 谓词   判断是否存在满足某种条件的记录
# 如果存在这样的记录就返回真（TRUE），如果不存在就返回假（FALSE）。EXISTS（存在）谓词的主语是“记录”，通常指定关联子查询作为EXIST的参数
# 可以把在 EXISTS 的子查询中书写 SELECT * 当作 SQL 的一种习惯。
SELECT  product_name, sale_price  FROM Product AS P 
WHERE 
EXISTS 
(SELECT *  FROM ShopProduct AS SP WHERE SP.shop_id = '000C'AND SP.product_id = P.product_id);

***********************************************************************************************************************************
# 使用CASE表达式进行行列转换
# 对按照商品种类计算出的销售单价合计值进行行列转换
SELECT 
SUM(CASE WHEN product_type = '衣服'      THEN sale_price ELSE 0 END) AS sum_price_clothes,
SUM(CASE WHEN product_type = '厨房用具'  THEN sale_price ELSE 0 END) AS sum_price_kitchen,
SUM(CASE WHEN product_type = '办公用品'  THEN sale_price ELSE 0 END) AS sum_price_office
FROM Product;

SELECT SUM(CASE WHEN sale_price <= 1000  THEN 1 ELSE 0 END) AS low_price,
SUM(CASE WHEN sale_price BETWEEN 1001 AND 3000THEN 1 ELSE 0 END) AS mid_price,
SUM(CASE WHEN sale_price >= 3001         THEN 1 ELSE 0 END) AS high_price
FROM Product;

***********************************************************************************************************************************
NOT IN 的参数中不能包含 NULL。不仅仅是指定 NULL 的情况，使用子查询作为 NOT IN 的参数时，该子查询的返回值也不能是 NULL

联结（JOIN）就是将其他表中的列添加过来，进行“添加列”的集合运算。
UNION是以行（纵向）为单位进行操作，而联结则是以列（横向）为单位进行的。

集合运算的优先级：intersect运算比union和except运算的优先级高，而union和except的优先级相等
UNION（并集）、INTERSECT（交集）、EXCEPT（差集）

# 两张表相等时返回“相等”，否则返回“不相等”
SELECT CASE WHEN COUNT(*) = 0
 THEN '相等'
 ELSE '不相等' END AS result
 FROM ((SELECT * FROM tbl_A
 UNION
 SELECT * FROM tbl_B)
 EXCEPT
 (SELECT * FROM tbl_A
 INTERSECT
 SELECT * FROM tbl_B)) TMP;

# 从表 EmpSkills中找出精通表 Skills中所有技术的员工。
# 用求差集的方法进行关系除法运算（有余数）
SELECT DISTINCT emp
 FROM EmpSkills ES1
 WHERE NOT EXISTS
 (SELECT skill
 FROM Skills
 EXCEPT
 SELECT skill
 FROM EmpSkills ES2
 WHERE ES1.emp = ES2.emp);
  
SELECT DISTINCT emp
 FROM EmpSkills ES1
 WHERE NOT EXISTS
 (SELECT skill
 FROM Skills
 EXCEPT
 SELECT skill
 FROM EmpSkills ES2
 WHERE ES1.emp = ES2.emp)
 AND NOT EXISTS
 (SELECT skill
 FROM EmpSkills ES3
 WHERE ES1.emp = ES3.emp
 EXCEPT
 SELECT skill
 FROM Skills );

SELECT emp
 FROM EmpSkills ES1
 WHERE NOT EXISTS
 (SELECT skill
 FROM Skills
 EXCEPT
 SELECT skill
 FROM EmpSkills ES2
 WHERE ES1.emp = ES2.emp)
 GROUP BY emp
HAVING COUNT(*) = (SELECT COUNT(*) FROM Skills);  
  

SELECT product_id, product_name
FROM Product
UNION
SELECT product_id, product_name
FROM Product2;

UNION
1.列数必须相同
2.列的类型必须一致
3.可以使用任何SELECT语句，但ORDER BY子句只能在最后使用一次

SELECT product_id, product_name
FROM Product
INTERSECT
SELECT product_id, product_name
FROM Product2
ORDER BY product_id;

SELECT COALESCE(SP.shop_id, '不确定') AS shop_id,
 COALESCE(SP.shop_name, '不确定') AS shop_name,
 P.product_id,
 P.product_name,
 P.sale_price
 FROM ShopProduct SP RIGHT OUTER JOIN Product P
 ON SP.product_id = P.product_id
ORDER BY shop_id;

***********************************************************************************************************************************
# 累加
SELECT product_id, product_name, sale_price,
SUM (sale_price) OVER (ORDER BY product_id) AS current_sum
FROM Product;


# 移动平均 
1.框架指定为“截止到之前 ~ 行”，因此“ROWS 2 PRECEDING”
KEYWORD 就是将框架指定为“截止到之前 2 行”，也就是将作为汇总对象的记录限定为如下的“最靠近的 3 行”。
● 自身（当前记录）
● 之前 1行的记录
● 之前 2行的记录
SELECT product_id, product_name, sale_price,
AVG (sale_price) OVER (ORDER BY product_id ROWS 2 PRECEDING) AS moving_avg
FROM Product;


2.截止到之后 ~ 行
SELECT product_id, product_name, sale_price,
AVG (sale_price) OVER (ORDER BY product_id ROWS 2 FOLLOWING) AS moving_avg
FROM Product;


3.将当前记录的前后行作为汇总对象
SELECT product_id, product_name, sale_price,
AVG (sale_price) OVER (ORDER BY product_id ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS moving_avg
FROM Product;


OVER 子句中的 ORDER BY 只是用来决定窗口函数按照什么样的顺序进行计算的，对结果的排列顺序并没有影响。
无法保证如下SELECT语句的结果的排列顺序
SELECT product_name, product_type, sale_price,
RANK () OVER (ORDER BY sale_price) AS ranking
FROM Product;

在语句末尾使用ORDER BY子句对结果进行排序
SELECT product_name, product_type, sale_price,
RANK () OVER (ORDER BY sale_price) AS ranking
FROM Product
ORDER BY ranking;




# 按照商品编号（product_id）的升序进行排序，计算出截至当前行的最高销售单价
SELECT product_id, product_name, sale_price,
MAX (sale_price) OVER (ORDER BY product_id) AS current_max_price
FROM Product;

# 计算出按照登记日期（regist_date）升序进行排列的各日期的销售单价（sale_price）的总额
SELECT regist_date, product_name, sale_price,
SUM (sale_price) OVER (ORDER BY regist_date NULLS FIRST) AS current_sum_price
FROM Product;


<窗口函数> OVER ([PARTITION BY <列清单>]
 ORDER BY <排序用列清单>)

① 能够作为窗口函数的聚合函数（SUM、AVG、COUNT、MAX、MIN）
② RANK、DENSE_RANK、ROW_NUMBER 等专用窗口函数


substr(to_char(a.transdate,'yyyy/mm/dd'),1,7)


TRUNCATE <表名>;只能删除表中全部数据，不能通过 WHERE 子句指定条件来删除部分数据。


select extract(year from charttime)-extract(year from dob) as 周岁 from  "脑脊液糖表查询出量"


创建表，携带数据   create table employees1 as select * from employees1 
创建表，携带表结构 create table employees2 like employees



SELECT pref_name,
-- 男性人口
SUM( CASE WHEN sex = '1' THEN population ELSE 0 END) AS cnt_m,
-- 女性人口
SUM( CASE WHEN sex = '2' THEN population ELSE 0 END) AS cnt_f
FROM PopTbl2
GROUP BY pref_name;


UPDATE Salaries
SET salary = 
CASE WHEN salary >= 300000 THEN salary * 0.9
WHEN salary >= 250000 AND salary < 280000  THEN salary * 1.2
ELSE salary END;


SELECT std_id,
CASE WHEN COUNT(*) = 1 THEN MAX(club_id)
ELSE 
MAX(CASE WHEN main_club_flg = 'Y' THEN club_id ELSE NULL END)
END AS main_club
FROM StudentClub GROUP BY std_id;


SELECT sex,
 SUM(population) AS total,
 SUM(CASE WHEN pref_name = '德岛' THEN population ELSE 0 END) AS col_1,
 SUM(CASE WHEN pref_name = '香川' THEN population ELSE 0 END) AS col_2,
 SUM(CASE WHEN pref_name = '爱媛' THEN population ELSE 0 END) AS col_3,
 SUM(CASE WHEN pref_name = '高知' THEN population ELSE 0 END) AS col_4,
 SUM(CASE WHEN pref_name IN ('德岛', '香川', '爱媛', '高知')
 THEN population ELSE 0 END) AS zaijie
 FROM PopTbl2
 GROUP BY sex;

# Oracle和mysql中 使用 GREATEST和LEAST函数来查找多列中最大和最小的值，
# 如果任何参数为NULL，则GREATEST和LEAST函数都返回NULL，可以使用IFNULL函数将NULL视为零来执行数字比较。

SELECT key, GREATEST(GREATEST(x,y), z) AS greatest FROM Greatests;

SELECT 
    company_id,
    LEAST(IFNULL(q1, 0),
            IFNULL(q2, 0),
            IFNULL(q3, 0),
            IFNULL(q4, 0)) low,
    GREATEST(IFNULL(q1, 0),
            IFNULL(q2, 0),
            IFNULL(q3, 0),
            IFNULL(q4, 0)) high
FROM
    revenues;

***********************************************************************************************************************************
可重排列、排列、组合
-- 用于获取可重排列的 SQL 语句
SELECT P1.name AS name_1, P2.name AS name_2
FROM Products P1, Products P2;

-- 用于获取排列的 SQL 语句
SELECT P1.name AS name_1, P2.name AS name_2
FROM Products P1, Products P2
WHERE P1.name <> P2.name;

--可重组合
SELECT P1.name AS name_1, P2.name AS name_2
 FROM Products P1, Products P2
 WHERE P1.name >= P2.name;
 

-- 用于获取组合的 SQL 语句
SELECT P1.name AS name_1, P2.name AS name_2
FROM Products P1, Products P2
WHERE P1.name > P2.name;

-- 用于获取组合的 SQL 语句 ：扩展成 3 列
SELECT P1.name AS name_1, P2.name AS name_2, P3.name AS name_3
FROM Products P1, Products P2, Products P3
WHERE P1.name > P2.name
AND P2.name > P3.name;

***********************************************************************************************************************************
ctid表示数据行在它所处的表内的物理位置，ctid字段的类型是tid。
尽管ctid可以快速定位数据行，每次vacuum full之后，数据行在块内的物理位置就会移动，即ctid会发生变化，所以ctid不能作为长期的行标识符，应该使用主键来标识一个逻辑行。
select  ctid  from analysis_taobao.bt_taobao_product_china_all_detail_2018 limit 30

select ctid,id from t1 where ctid='(0,11)';

利用ctid删除重复数据
delete from t1 a where a.ctid <>(select min(b.ctid) from t1 b where a.id=b.id);

***********************************************************************************************************************************
# 求中位数的 SQL 语句 ：在 HAVING 子句中使用非等值自连接
SELECT AVG(DISTINCT income)
 FROM (SELECT T1.income
 FROM Graduates T1, Graduates T2
 GROUP BY T1.income
#  S1 的条件
 HAVING SUM(CASE WHEN T2.income >= T1.income THEN 1 ELSE 0 END)
 >= COUNT(*) / 2
#  S2 的条件
 AND SUM(CASE WHEN T2.income <= T1.income THEN 1 ELSE 0 END)
 >= COUNT(*) / 2 ) TMP;

# 带余除法,查询啤酒、纸尿裤和自行车同时在库的店铺  
SELECT SI.shop
 FROM ShopItems SI, Items I
 WHERE SI.item = I.item
 GROUP BY SI.shop
HAVING COUNT(SI.item) = (SELECT COUNT(item) FROM Items);


# 精确关系除法运算 ：使用外连接和 COUNT 函数
SELECT SI.shop
 FROM ShopItems SI LEFT OUTER JOIN Items I
 ON SI.item=I.item
 GROUP BY SI.shop
HAVING COUNT(SI.item) = (SELECT COUNT(item) FROM Items)　　-- 条件 1
 AND COUNT(I.item) = (SELECT COUNT(item) FROM Items); -- 条件 2


SELECT CASE WHEN COUNT(*) <> MAX(seq)
 THEN '存在缺失的编号'
 ELSE '不存在缺失的编号' END AS gap
 FROM SeqTbl;


-- 查找所有学生都在 9 月份提交完成的学院 (1)：使用 BETWEEN 谓词
SELECT dpt
 FROM Students
 GROUP BY dpt
HAVING COUNT(*) = SUM(CASE WHEN sbmt_date BETWEEN '2005-09-01' AND '2005-09-30'
 THEN 1 ELSE 0 END);


-- 查找所有学生都在 9 月份提交完成的学院 (1)：使用 EXTRACT 谓词
SELECT dpt
 FROM Students
 GROUP BY dpt
HAVING COUNT(*) = SUM(CASE WHEN EXTRACT (YEAR FROM sbmt_date) = 2005
 AND EXTRACT (MONTH FROM sbmt_date) = 09
 THEN 1 ELSE 0 END);

             
                 
SELECT name,
 CASE WHEN SUM(CASE WHEN course = 'SQL 入门' THEN 1 ELSE NULL END) = 1
 THEN '○' ELSE NULL END AS "SQL 入门",
 CASE WHEN SUM(CASE WHEN course = 'UNIX 基础' THEN 1 ELSE NULL END) = 1
 THEN '○' ELSE NULL END AS "UNIX 基础",
 CASE WHEN SUM(CASE WHEN course = 'Java 中级' THEN 1 ELSE NULL END) = 1
 THEN '○' ELSE NULL END AS "Java 中级 "
 FROM Courses
 GROUP BY name;             
             
                  
用外连接求差集 ：A － B             
SELECT A.id AS id, A.name AS A_name
 FROM Class_A A LEFT OUTER JOIN Class_B B
 ON A.id = B.id
 WHERE B.name IS NULL;             
             
用外连接求差集 ：B － A             
SELECT B.id AS id, B.name AS B_name
 FROM Class_A A RIGHT OUTER JOIN Class_B B
 ON A.id = B.id
 WHERE A.name IS NULL;             
             
             
用全外连接求异或集
SELECT COALESCE(A.id, B.id) AS id,
 COALESCE(A.name , B.name ) AS name
 FROM Class_A A FULL OUTER JOIN Class_B B
 ON A.id = B.id
 WHERE A.name IS NULL
 OR B.name IS NULL;             
             

             
用列表展示与上一年的比较结果             
-- 求出是增长了还是减少了，抑或是维持现状 (1)：使用关联子查询
SELECT S1.year, S1.sale,
 CASE WHEN sale =
 (SELECT sale
 FROM Sales S2
 WHERE S2.year = S1.year - 1) THEN '→' -- 持平
 WHEN sale >
 (SELECT sale
 FROM Sales S2
 WHERE S2.year = S1.year - 1) THEN '↑' -- 增长
 WHEN sale <
 (SELECT sale
 FROM Sales S2
 WHERE S2.year = S1.year - 1) THEN '↓' -- 减少
 ELSE '—' END AS var
 FROM Sales S1
 ORDER BY year;



-- 求出是增长了还是减少了，抑或是维持现状 (2)：使用自连接查询（最早的年份不会出现在结果里）
SELECT S1.year, S1.sale,
 CASE WHEN S1.sale = S2.sale THEN '→'
 WHEN S1.sale > S2.sale THEN '↑'
 WHEN S1.sale < S2.sale THEN '↓'
 ELSE '—' END AS var
 FROM Sales S1, Sales S2
 WHERE S2.year = S1.year - 1
 ORDER BY year;

符号函数sign
当x>0，sign(x)=1;
当x=0，sign(x)=0;
当x<0， sign(x)=-1；

-- 一次性求出增长、减退、维持现状的状态 (3)：使用 SIGN 函数
SELECT S1.year, S1.sale,
 CASE SIGN(sale -
 (SELECT sale
 FROM Sales S2
 WHERE S2.year = S1.year - 1) )
 WHEN 0 THEN '→' -- 持平
 WHEN 1 THEN '↑' -- 增长
 WHEN -1 THEN '↓' -- 减少
 ELSE ' - ' END AS var
 FROM Sales S1
 ORDER BY year;

***********************************************************************************************************************************
时间轴有间断时 ：和过去最临近的时间进行比较
1. 与该年份相比是过去的年份。
2. 在满足条件 1 的年份中，年份最早的一个。

-- 查询与过去最临近的年份营业额相同的年份
SELECT year, sale
 FROM Sales2 S1
 WHERE sale =
 (SELECT sale
 FROM Sales2 S2
 WHERE S2.year =
 (SELECT MAX(year) -- 条件 2 ：在满足条件 1 的年份中，年份最早的一个
 FROM Sales2 S3
 WHERE S1.year > S3.year)) -- 条件１ ：与该年份相比是过去的年份
 ORDER BY year;


查询与过去最临近的年份营业额相同的年份  
SELECT S1.year AS year,
 S1.year AS year
 FROM Sales2 S1, Sales2 S2
 WHERE S1.sale = S2.sale
 AND S2.year = (SELECT MAX(year)
 FROM Sales2 S3
 WHERE S1.year > S3.year)
 ORDER BY year;


-- 求每一年与过去最临近的年份之间的营业额之差 (1)：结果里不包含最早的年份
SELECT S2.year AS pre_year,
 S1.year AS now_year,
 S2.sale AS pre_sale,
 S1.sale AS now_sale,
 S1.sale - S2.sale AS diff
 FROM Sales2 S1, Sales2 S2
 WHERE S2.year = (SELECT MAX(year)
 FROM Sales2 S3
 WHERE S1.year > S3.year)
 ORDER BY now_year;

-- 求每一年与过去最临近的年份之间的营业额之差 (2)：使用自外连接。结果里包含最早的年份
SELECT S2.year AS pre_year, S1.year AS now_year,
 S2.sale AS pre_sale, S1.sale AS now_sale,
 S1.sale - S2.sale AS diff
 FROM Sales2 S1 LEFT OUTER JOIN Sales2 S2
 ON S2.year = (SELECT MAX(year)
 FROM Sales2 S3
 WHERE S1.year > S3.year)
 ORDER BY now_year;

***********************************************************************************************************************************
alter table hengshui alter COLUMN shop_name_ type VARCHAR(150)

查询学生姓名，如果学生名字为null或''则显示“姓名为空”
select case when coalesce(name,'') = '' then '姓名为空' else name end  from student;


-- 求重叠的住宿期间
SELECT reserver, start_date, end_date
 FROM Reservations R1
 WHERE EXISTS
 (SELECT *
 FROM Reservations R2
 WHERE R1.reserver <> R2.reserver -- 与自己以外的客人进行比较
 AND ( R1.start_date BETWEEN R2.start_date AND R2.end_date
 -- 条件 (1)：自己的入住日期在他人的住宿期间内
 OR R1.end_date BETWEEN R2.start_date AND R2.end_date));
 -- 条件 (2)：自己的离店日期在他人的住宿期间内


-- 升级版 ：把完全包含别人的住宿期间的情况也输出
SELECT reserver, start_date, end_date
 FROM Reservations R1
WHERE EXISTS
 (SELECT *
 FROM Reservations R2
 WHERE R1.reserver <> R2.reserver
 AND ( ( R1.start_date BETWEEN R2.start_date
 AND R2.end_date
 OR R1.end_date BETWEEN R2.start_date
 AND R2.end_date)
 OR ( R2.start_date BETWEEN R1.start_date
 AND R1.end_date
 AND R2.end_date BETWEEN R1.start_date
 AND R1.end_date)));


-- 生成供应商的全部组合
SELECT SP1.sup AS s1, SP2.sup AS s2
 FROM SupParts SP1, SupParts SP2
 WHERE SP1.sup < SP2.sup
 GROUP BY SP1.sup, SP2.sup;


SELECT SP1.sup AS s1, SP2.sup AS s2
 FROM SupParts SP1, SupParts SP2
 WHERE SP1.sup < SP2.sup -- 生成供应商的全部组合
 AND SP1.part = SP2.part -- 条件 1 ：经营同种类型的零件
 GROUP BY SP1.sup, SP2.sup
HAVING COUNT(*) = (SELECT COUNT(*) -- 条件 2 ：经营的零件种类数相同
 FROM SupParts SP3
 WHERE SP3.sup = SP1.sup)
 AND COUNT(*) = (SELECT COUNT(*)
 FROM SupParts SP4
 WHERE SP4.sup = SP2.sup);

***********************************************************************************************************************************
用于删除重复行的高效 SQL
-- 删除重复行 ：使用关联子查询
DELETE FROM Products
 WHERE rowid < ( SELECT MAX(P2.rowid)
 FROM Products P2
 WHERE Products.name = P2. name
 AND Products.price = P2.price ) ;

-- 用于删除重复行的高效 SQL 语句 (1)：通过 EXCEPT 求补集
DELETE FROM Products
 WHERE rowid IN ( SELECT rowid -- 全部 rowid
 FROM Products
 EXCEPT -- 减去
 SELECT MAX(rowid) -- 要留下的 rowid
 FROM Products
 GROUP BY name, price) ;


-- 删除重复行的高效 SQL 语句 (2)：通过 NOT IN 求补集
DELETE FROM Products
 WHERE rowid NOT IN ( SELECT MAX(rowid)
 FROM Products
 GROUP BY name, price);

***********************************************************************************************************************************
判断两张表 UNION 之后的结果与原来的两张表行数是否相等
SELECT CASE WHEN COUNT(*) = (SELECT COUNT(*) FROM tbl_A )
 AND COUNT(*) = (SELECT COUNT(*) FROM tbl_B )
 THEN '相等'
 ELSE '不相等' END AS result
 FROM ( SELECT *
 FROM tbl_A
 UNION
 SELECT *
 FROM tbl_B ) TMP;



EXISTS参数是行数据的集合
           
             
 CREATE TABLE test4 (  
     id BIGINT UNSIGNED  PRIMARY KEY AUTO_INCREMENT,  
     brand VARCHAR(255) NOT NULL,  
     color ENUM('RED','GREEN','BLUE')
  ) ENGINE = InnoDB; 



SELECT student_id
 FROM TestScores TS1
 WHERE subject IN ('数学', '语文')
 AND NOT EXISTS
 (SELECT *
 FROM TestScores TS2
 WHERE TS2.student_id = TS1.student_id
 AND 1 = CASE WHEN subject = '数学' AND score < 80 THEN 1
 WHEN subject = '语文' AND score < 50 THEN 1
 ELSE 0 END)
 GROUP BY student_id
HAVING COUNT(*) = 2; -- 必须两门科目都有分数



-- 查询完成到了工程 1 的项目 ：面向集合的解法
SELECT project_id
 FROM Projects
 GROUP BY project_id
HAVING COUNT(*) = SUM(CASE WHEN step_nbr <= 1 AND status = '完成' THEN 1
 WHEN step_nbr > 1 AND status = '等待' THEN 1
 ELSE 0 END);

            
            
 -- 查询完成到了工程 1 的项目 ：谓词逻辑的解法
SELECT *
 FROM Projects P1
 WHERE NOT EXISTS
 (SELECT status
 FROM Projects P2
 WHERE P1.project_id = P2. project_id -- 以项目为单位进行条件判断
 AND status <> CASE WHEN step_nbr <= 1 -- 使用双重否定来表达全称量化命题
 THEN '完成'
 ELSE '等待' END);           
            
                    
            
1. 查询“都是 1”的行

SELECT *
 FROM ArrayTbl
 WHERE 1 = ALL (col1, col2, col3, col4, col5, col6, col7, col8, col9, col10);
 
2. 查询“至少有一个 9”的行
            
SELECT *
 FROM ArrayTbl
 WHERE 9 = ANY (col1, col2, col3, col4, col5, col6, col7, col8, col9, col10);            
            
            
-- 查询全是 NULL 的行 ：正确的解法
SELECT *
 FROM ArrayTbl
 WHERE COALESCE(col1, col2, col3, col4, col5, col6, col7, col8, col9, col10) IS NULL;            
            
求所有的缺失编号           
SELECT seq
 FROM Sequence N
 WHERE seq BETWEEN 1 AND 12
 AND NOT EXISTS
 (SELECT *
 FROM SeqTbl S
 WHERE N.seq = S.seq );            
    
SELECT N.seq
 FROM Sequence N LEFT OUTER JOIN SeqTbl S
 ON N.seq = S.seq
 WHERE N.seq BETWEEN 1 AND 12
 AND S.seq IS NULL;

1. 首选方法：EXCEPT
2. 不支持 EXCEPT 的数据库也能使用，而且易于理解的方法：NOT IN
3. NOT IN 的相似方法：NOT EXISTS
4. 麻烦的方法：外连接


SELECT S1.seat AS start_seat,
 S2.seat AS end_seat,
 S2.seat - S1.seat + 1 AS seat_cnt
 FROM Seats3 S1, Seats3 S2, Seats3 S3
 WHERE S1.seat <= S2.seat -- 第一步 ：生成起点和终点的组合
 AND S3.seat BETWEEN S1.seat - 1 AND S2.seat + 1
GROUP BY S1.seat, S2.seat
HAVING COUNT(*) = SUM(CASE WHEN S3.seat BETWEEN S1.seat AND S2.seat
 AND S3.status = '空' THEN 1 -- 条件 1
 WHEN S3.seat = S2.seat + 1 AND S3.status = '占' THEN 1 -- 条件 2
 WHEN S3.seat = S1.seat - 1 AND S3.status = '占' THEN 1 -- 条件 3
 ELSE 0 END);

***********************************************************************************************************************************
所有队员都处于‘待命’状态      “所有队员都处于待命状态”＝“不存在不处于待命状态的队员”
-- 用谓词表达全称量化命题
SELECT team_id, member
 FROM Teams T1
 WHERE NOT EXISTS
 (SELECT *
 FROM Teams T2
 WHERE T1.team_id = T2.team_id
 AND status <> '待命' );

-- 用集合表达全称量化命题 (1)
SELECT team_id
 FROM Teams
 GROUP BY team_id
HAVING COUNT(*) = SUM(CASE WHEN status = '待命'
 THEN 1
 ELSE 0 END);


-- 用集合表达全称量化命题 (2)
SELECT team_id
 FROM Teams
 GROUP BY team_id
HAVING MAX(status) = '待命'
 AND MIN(status) = '待命';



-- 选中材料存在重复的生产地
SELECT center
 FROM Materials
 GROUP BY center
HAVING COUNT(material) <> COUNT(DISTINCT material);

SELECT center,
 CASE WHEN COUNT(material) <> COUNT(DISTINCT material) THEN '存在重复'
 ELSE '不存在重复' END AS status
 FROM Materials
 GROUP BY center;

-- 存在重复的集合 ：使用 EXISTS
SELECT center, material
 FROM Materials M1
 WHERE EXISTS
 (SELECT *
 FROM Materials M2
 WHERE M1.center = M2.center
 AND M1.receive_date <> M2.receive_date
 AND M1.material = M2.material);


-- 如果有查询结果，说明存在缺失的编号
SELECT '存在缺失的编号' AS gap
 FROM SeqTbl
HAVING COUNT(*) <> MAX(seq);


-- 如果有查询结果，说明存在缺失的编号 ：只调查数列的连续性
SELECT '存在缺失的编号' AS gap
 FROM SeqTbl
HAVING COUNT(*) <> MAX(seq) - MIN(seq) + 1 ;


-- 不论是否存在缺失的编号都返回一行结果
SELECT CASE WHEN COUNT(*) = 0
 THEN '表为空'
 WHEN COUNT(*) <> MAX(seq) - MIN(seq) + 1
 THEN '存在缺失的编号'
 ELSE '连续' END AS gap
 FROM SeqTbl;



-- 查找最小的缺失编号 ：表中没有 1 时返回 1
SELECT CASE WHEN COUNT(*) = 0 OR MIN(seq) > 1 -- 最小值不是 1 时→返回 1
 THEN 1
 ELSE (SELECT MIN(seq +1) -- 最小值是1时→返回最小的缺失编号
 FROM SeqTbl S1
 WHERE NOT EXISTS
 (SELECT *
 FROM SeqTbl S2
 WHERE S2.seq = S1.seq + 1)) END
 FROM SeqTbl;

***********************************************************************************************************************************
查询出 75% 以上的学生分数都在 80 分以上的班级
 SELECT class
 FROM TestResults
GROUP BY class
 HAVING COUNT(*) * 0.75
 <= SUM(CASE WHEN score >= 80
 THEN 1
 ELSE 0 END) ;
        
查询出分数在 50 分以上的男生的人数比分数在 50 分以上的女生的人数多的班级       
SELECT class
 FROM TestResults
GROUP BY class
 HAVING SUM(CASE WHEN score >= 50 AND sex = '男'
 THEN 1
 ELSE 0 END)
 > SUM(CASE WHEN score >= 50 AND sex = '女'
 THEN 1
 ELSE 0 END) ;       
        
 查询出女生平均分比男生平均分高的班级            
-- 比较男生和女生平均分的 SQL 语句 (1)：对空集使用 AVG 后返回 0
 SELECT class
 FROM TestResults
GROUP BY class
 HAVING AVG(CASE WHEN sex = '男'
 THEN score
 ELSE 0 END)
 < AVG(CASE WHEN sex = '女'
 THEN score
 ELSE 0 END) ;        
        
        
-- 比较男生和女生平均分的 SQL 语句 (2)：对空集求平均值后返回 NULL
 SELECT class
 FROM TestResults
GROUP BY class
 HAVING AVG(CASE WHEN sex = '男'
 THEN score
 ELSE NULL END)
 < AVG(CASE WHEN sex = '女'
 THEN score
 ELSE NULL END) ;        
        
               
■ 用于调查集合性质的常用条件及其用途
No 条件表达式 用途
1 COUNT (DISTINCT col) = COUNT (col) col 列没有重复的值
2 COUNT(*) = COUNT(col) col 列不存在 NULL
3 COUNT(*) = MAX(col) col 列是连续的编号（起始值是 1）
4 COUNT(*) = MAX(col) - MIN(col) + 1 col 列是连续的编号（起始值是任意整数）
5 MIN(col) = MAX(col) col 列都是相同值，或者是 NULL
6 MIN(col) * MAX(col) > 0 col 列全是正数或全是负数
7 MIN(col) * MAX(col) < 0 col 列的最大值是正数，最小值是负数
8 MIN(ABS(col)) = 0 col 列最少有一个是 0
9 MIN(col - 常量 ) = - MAX(col - 常量 ) col 列的最大值和最小值与指定常量等距
        
        
        
-- 选择（材料，原产国）组合有重复的生产地
SELECT center
 FROM Materials2
 GROUP BY center
HAVING COUNT(material || orgland) <> COUNT(DISTINCT material || orgland);       
        
        
 SELECT student_id
 FROM TestScores
 WHERE subject IN ('数学', '语文')
 GROUP BY student_id
HAVING SUM(CASE WHEN subject = '数学' AND score >= 80 THEN 1
 WHEN subject = '语文' AND score >= 50 THEN 1
 ELSE 0 END) = 2;       
        
        
        
        
        

In [ ]:
/**gp中的基本sql语法**/
--删除表
drop table testtb;
--创建表
CREATE TABLE testtb
(
  id integer,
  "name" character varying(2)
)
WITH (
  OIDS=FALSE
)
DISTRIBUTED BY (id);
ALTER TABLE testtb OWNER TO gpadmin;
--WITH 用来说明表的存储属性，比如表的压缩
--DISTRIBUTED BY 定义表的分布键，这个键最好唯一，如果表中没有唯一键字段，可以定义DISTRIBUTED RANDOMLY作为分布建。

--表的创建
create table testTBbak (like testTB)                                  -- like 复制表结构
create table testtbbak1 as select * from testtb distributed by (name) -- 数据保留，可以指定分布键。
select * into testtbbak2 from testTB                                  -- into 保留数据，不能指定分布键
select * from testtbbak2


--创建视图
create view v_testtb as select * from testtb

-- 删除数据
delete from testtb -- 速度慢，会写日志。
truncate testtb    -- 速度快，但不会写日期。

/**序列的使用**/
--删除序列
drop sequence myseq
--创建序列（1是开始值，可以根据实际情况改变）
create sequence myseq start 1
--序列的使用
insert into testTB values(nextval('myseq'),'小明')
--重新设置序列值
select setval('myseq',1)
--查询
select id,name from testTB order by 1

-- explain 用于查看sql执行计划
explain select * from testtb 

/**字符串函数**/
-- || 字符串连接
select '胡' || '黄' || '腾'
-- length 字符长度
select length('89998')
-- substring 字符串截取,从那位开始，截几位
select substring('123456' from 2 for 3)
-- trim去字符前后空格
select trim(' 234 fds   ')
-- lower 字符转小写
select lower('RER')
-- upper 字符转大写
select upper('rer')
-- replace 替换字符
select replace('aabbddee','aa','mm')
-- position 查找指定字符位置
select position('de' in 'abdeaf')
-- split_part 根据指定字符查找给定字符的位置
select split_part('aaa|bbb|ccc|eee','|',2)

/**时间的函数**/
-- age求俩日期之差
select age(timestamp '1987-12-03',timestamp '2015-06-17')
-- age如果一个日期则和当前日期求差
select age(timestamp '1987-12-03')
-- current_date 当前日期
select current_date
-- current_time 当前时间
select current_time
-- current_timestamp 当前时间戳
select current_timestamp
-- now 当前时间
select now()
-- extract 获取指定时间的具体某个参数
select extract(day from date '2015-06-18')
select extract(day from now())
-- 时间相加
select '2015-04-2 10:00:52'::timestamp + interval '10 days 2 hours 10 seconds' 
-- 时间相减
select current_date - interval '10 days'



/**gp中其他函数**/
-- greatest 取两值中的最大值
select greatest(3,9)
-- 列转行
insert into testtb(id,name)values (1,'ha');
insert into testtb(id,name)values (1,'he');
insert into testtb(id,name)values (1,'hi');
insert into testtb(id,name)values (2,'xb');
insert into testtb(id,name)values (2,'xh');
insert into testtb(id,name)values (2,'xm');
select id,string_agg(name,'|' order by name) from testtb group by id;
-- 行转列
select id,regexp_split_to_table(string_agg,E'\\|') str from texttb_m

/**开窗函数**/
--建表
create table empsalary
(
  depname varchar(20),
  empno integer,
  salary integer
) 
distributed by (empno)

insert into empsalary values ('develop',9,4500);
insert into empsalary values ('develop',1,3200);
insert into empsalary values ('develop',4,1000);
insert into empsalary values ('develop',2,9100);
insert into empsalary values ('develop',6,1000);
insert into empsalary values ('person1',5,3100);
insert into empsalary values ('person1',7,4100);
insert into empsalary values ('sales',3,2400);
insert into empsalary values ('sales',8,1200);
insert into empsalary values ('sales',10,5100);

-- rank 及 row_number 函数的应用
select depname
       ,empno
       ,salary
       ,rank() over (partition by depname order by salary desc)
       ,row_number() over (partition by depname order by salary desc)
from empsalary


select *
       ,sum(salary) over () sum1
       ,sum(salary) over (order by salary) sum2
       ,sum(salary) over (partition by depname) sum3
       ,sum(salary) over (partition by depname order by salary) sum4
 from empsalary

-- grouping by 的使用（其实就是简化了union）
select depname ,sum(empno)
from empsalary
group by depname
union all
select depname ,sum(salary)
from empsalary
group by depname

--等效于
select depname,sum(empno),sum(salary)
from empsalary 
group by grouping sets(depname)

# 分组后合计
SELECT product_type, regist_date, SUM(sale_price) AS sum_price FROM Product
GROUP BY ROLLUP(product_type, regist_date);


GROUPING SETS 列出所有你设置的分组集
CUBE          列出所有可能的分组集
ROOLUP        以层级的方式列出分组集







In [ ]:
create table dm_user_quarter_pay_summary as
	select case when ds>='20180101' and ds<='20180331' then '2018_1'
	            when ds>='20181001' and ds<='20181231' then '2018_4'
	            when ds>='20190101' and ds<='20190331' then '2019_1'
	            when ds>='20190401' and ds<='20190520' then '2019_2'
	       end as qtr
           ,case when product_id like 'vip%' then 'vip'
                 when product_id like 'coin%' then 'coin'
                 when product_id like 'gem%' then 'gem'
           end as pay_type
           ,sum(pay_money) as pay_money
           ,user_id
	from chuman_bi.chuman_tlog_pay
	where ((ds>='20180101' and ds<='20180331') or (ds>='20181001' and ds<='20190520')) and (product_id like 'vip%' or product_id like 'coin%' or product_id like 'gem%')
	group by case when ds>='20180101' and ds<='20180331' then '2018_1'
	            when ds>='20181001' and ds<='20181231' then '2018_4'
	            when ds>='20190101' and ds<='20190331' then '2019_1'
	            when ds>='20190401' and ds<='20190520' then '2019_2'
	       end
           ,case when product_id like 'vip%' then 'vip'
                 when product_id like 'coin%' then 'coin'
                 when product_id like 'gem%' then 'gem'
           end
           ,user_id
;

create table dm_user_quarter_level as
select user_id,
       case when pay_money_2018_1>= 100 then 1
            when pay_money_2018_1>= 20 and pay_money_2018_1<100 then 2
            when pay_money_2018_1>0 and pay_money_2018_1<20 then 3
       end as user_level_2018_1,
       case when pay_money_2018_4>= 100 then 1
            when pay_money_2018_4>= 20 and pay_money_2018_4<100 then 2
            when pay_money_2018_4>0 and pay_money_2018_4<20 then 3
       end as user_level_2018_4,
       case when pay_money_2019_1>= 100 then 1
            when pay_money_2019_1>= 20 and pay_money_2019_1<100 then 2
            when pay_money_2019_1>0 and pay_money_2019_1<20 then 3
       end as pay_money_2019_1,
       case when pay_money_2019_2>= 100 then 1
            when pay_money_2019_2>= 20 and pay_money_2019_2<100 then 2
            when pay_money_2019_2>0 and pay_money_2019_2<20 then 3
       end as pay_money_2019_2
from (
	select user_id
	       ,sum(case when qtr='2018_1' then pay_money end) pay_money_2018_1
	       ,sum(case when qtr='2018_4' then pay_money end) pay_money_2018_4
	       ,sum(case when qtr='2019_1' then pay_money end) pay_money_2019_1
	       ,sum(case when qtr='2019_2' then pay_money end) pay_money_2019_2
	 from dm_user_quarter_pay_summary group by user_id
) a
;



---用户类型
with tb1 as
(
select case when substr(created,1,10)>='2018-01-01' and substr(created,1,10)<='2018-03-31' then 4
	        when substr(created,1,10)>='2018-10-01' and substr(created,1,10)<='2018-12-31' then 3
	        when substr(created,1,10)>='2019-01-01' and substr(created,1,10)<='2019-03-31' then 2
	        when substr(created,1,10)>='2019-04-01' and substr(created,1,10)<='2019-05-20' then 1
	        else 0
	   end as reg_qtr
       ,case when to_char(from_unixtime(last_login_time),'yyyy-mm-dd') < '2019-05-01' then 1 else 0 end as is_lost
       ,id as user_id
from chuman_big_data_dev.ods_gugu_user_all
where ds='20190522' and substr(created,1,10)<='2019-05-20'
),

---消费
tb2 as (
select user_id
       ,sum(case when type='钻石' and log_type=3 then num end) as gem_3
       ,sum(case when type='钻石' and log_type=4 then num end) as gem_4
       ,sum(case when type='钻石' and log_type=5 then num end) as gem_5
       ,sum(case when type='钻石' and log_type=10 then num end) as gem_10
       ,sum(case when type='钻石' and log_type=15 then num end) as gem_15
       ,sum(case when type='钻石' and log_type=2 then num end) as gem_2
       ,sum(case when type='钻石' and log_type=21 then num end) as gem_21
       ,sum(case when type='钻石' and log_type=17 then num end) as gem_17
       ,sum(case when type='钻石' and log_type=18 then num end) as gem_18
       ,sum(case when type='钻石' and log_type=19 then num end) as gem_19
       ,sum(case when type='钻石' and log_type=12 then num end) as gem_12
       ,sum(case when type='钻石' and log_type=26 then num end) as gem_26
       ,sum(case when type='金币' and log_type=5 then num end) as coin_5
       ,sum(case when type='金币' and log_type=8 then num end) as coin_8
       ,sum(case when type='金币' and log_type=14 then num end) as coin_14
       ,sum(case when type='金币' and log_type=16 then num end) as coin_16
       ,sum(case when type='金币' and log_type=2 then num end) as coin_2
       ,sum(case when type='金币' and log_type=1 then num end) as coin_1
       ,sum(case when type='金币' and log_type=4 then num end) as coin_4
       ,sum(case when type='钻石' then num end) tot_gem
       ,sum(case when type='金币' then num end) tot_coin
  from chuman_bi.chuman_consume_log
 where ds >='20190401' and ds<='20190520'
   and ((type='钻石' and log_type in (2,3,4,5,10,12,15,17,18,19,21,26)) or (type='金币' and log_type in (1,2,4,5,8,14,16)))
 group by user_id
),

---充值
tb3 as (
select user_id,sum(case when pay_type='coin' then pay_money end) coin_pay ,sum(case when pay_type='vip' then pay_money end) vip_pay ,sum(case when pay_type='gem' then pay_money end) gem_pay 
  from chuman_big_data_dev.dm_user_quarter_pay_summary
  where qtr='2019_2'
  group by user_id
)

select reg_qtr
       ,is_lost
       ,user_level_2018_1
       ,user_level_2018_4
       ,user_level_2019_1
       ,user_level_2019_2
       ,is_gem_pay
       ,is_coin_pay
       ,is_vip_pay
       ,count(distinct user_id) user_num
       ,sum(total_pay) total_pay
       ,sum(gem_pay) gem_pay
       ,sum(coin_pay) coin_pay
       ,sum(vip_pay) vip_pay
       ,sum(gem_3) gem_3
       ,sum(gem_4) gem_4
       ,sum(gem_5) gem_5
       ,sum(gem_10) gem_10
       ,sum(gem_15) gem_15
       ,sum(gem_2) gem_2
       ,sum(gem_21) gem_21
       ,sum(gem_17) gem_17
       ,sum(gem_18) gem_18
       ,sum(gem_19) gem_19
       ,sum(gem_12) gem_12
       ,sum(gem_26) gem_26
       ,sum(coin_5) coin_5
       ,sum(coin_8) coin_8
       ,sum(coin_14) coin_14
       ,sum(coin_16) coin_16
       ,sum(coin_2) coin_2
       ,sum(coin_1) coin_1
       ,sum(coin_4) coin_4
       ,sum(tot_gem) tot_gem
       ,sum(tot_coin) tot_coin
from 
(
	select ta.user_id
	       ,ta.reg_qtr
	       ,ta.is_lost
	       ,coalesce(td.user_level_2018_1,0) user_level_2018_1
	       ,coalesce(td.user_level_2018_4,0) user_level_2018_4
	       ,coalesce(td.pay_money_2019_1,0) user_level_2019_1
	       ,coalesce(td.pay_money_2019_2,0) user_level_2019_2
	       ,case when tc.gem_pay>0 then 1 else 0 end as is_gem_pay
	       ,case when tc.coin_pay>0 then 1 else 0 end as is_coin_pay
	       ,case when tc.vip_pay>0 then 1 else 0 end as is_vip_pay
	       ,(coalesce(tc.gem_pay,0)+coalesce(tc.coin_pay,0)+coalesce(tc.vip_pay,0)) total_pay
	       ,tc.gem_pay
	       ,tc.coin_pay
	       ,tc.vip_pay
	       ,tb.gem_3
	       ,tb.gem_4
	       ,tb.gem_5
	       ,tb.gem_10
		   ,tb.gem_15
		   ,tb.gem_2
		   ,tb.gem_21
		   ,tb.gem_17
		   ,tb.gem_18
		   ,tb.gem_19
		   ,tb.gem_12
		   ,tb.gem_26
		   ,tb.coin_5
		   ,tb.coin_8
		   ,tb.coin_14
		   ,tb.coin_16
		   ,tb.coin_2
		   ,tb.coin_1
		   ,tb.coin_4
		   ,tb.tot_gem
		   ,tb.tot_coin
	  from tb1 ta
	  left join tb2 tb
	  on ta.user_id=tb.user_id
	  left join tb3 tc
	  on ta.user_id=tc.user_id
	  left join chuman_big_data_dev.dm_user_quarter_level td
	  on ta.user_id=td.user_id
) a
group by reg_qtr,is_lost,user_level_2018_1,user_level_2018_4,user_level_2019_1,user_level_2019_2,is_gem_pay,is_coin_pay,is_vip_pay

In [ ]:
CREATE TABLE "gather_taobao"."ir_taobao_product_trade_china_2019" (
"id" int8 DEFAULT nextval('"gather_taobao".ir_taobao_product_trade_china_2019_id_seq'::regclass) NOT NULL,
"prod_id" int8,
"prod_name" varchar(255) COLLATE "default",
"url" varchar(255) COLLATE "default",
"shop_id" int8,
"shop_name" varchar(255) COLLATE "default",
"type" varchar(50) COLLATE "default",
"company" varchar(255) COLLATE "default",
"first_areaid" int8,
"second_areaid" int8,
"third_areaid" int8,
"address" varchar(255) COLLATE "default",
"first_addr" varchar(50) COLLATE "default",
"second_addr" varchar(50) COLLATE "default",
"third_addr" varchar(50) COLLATE "default",
"cid" int8,
"prod_firstcat" varchar(100) COLLATE "default",
"prod_seccat" varchar(50) COLLATE "default",
"prod_sort" varchar(100) COLLATE "default",
"brand_id" int8,
"brand_name" varchar(100) COLLATE "default",
"agri_keyword" varchar(100) COLLATE "default",
"agri_keyid" int8,
"agri_firstcat" varchar(100) COLLATE "default",
"agri_seccat" varchar(100) COLLATE "default",
"agri_thirdcat" varchar(100) COLLATE "default",
"unit_price" numeric(16,4),
"unit" varchar(100) COLLATE "default",
"price_01" numeric(16,4),
"month_amount_01" int8,
"month_money_01" numeric(16,4),
"price_02" numeric(16,4),
"month_amount_02" int8,
"month_money_02" numeric(16,4),
"price_03" numeric(16,4),
"month_amount_03" int8,
"month_money_03" numeric(16,4),
"price_04" numeric(16,4),
"month_amount_04" int8,
"month_money_04" numeric(16,4),
"price_05" numeric(16,4),
"month_amount_05" int8,
"month_money_05" numeric(16,4),
"price_06" numeric(16,4),
"month_amount_06" int8,
"month_money_06" numeric(16,4),
"price_07" numeric(16,4),
"month_amount_07" int8,
"month_money_07" numeric(16,4),
"price_08" numeric(16,4),
"month_amount_08" int8,
"month_money_08" numeric(16,4),
"price_09" numeric(16,4),
"month_amount_09" int8,
"month_money_09" numeric(16,4),
"price_10" numeric(16,4),
"month_amount_10" int8,
"month_money_10" numeric(16,4),
"price_11" numeric(16,4),
"month_amount_11" int8,
"month_money_11" numeric(16,4),
"price_12" numeric(16,4),
"month_amount_12" int8,
"month_money_12" numeric(16,4),
"createtime" timestamp(6),
"creator" varchar(50) COLLATE "default",
"modifitime" timestamp(6),
"modifier" varchar(50) COLLATE "default",
"yc_brand_id" int4,
"brand_cname" varchar(255) COLLATE "default",
"areabrand_id" int4,
"areabrand_name" varchar(255) COLLATE "default",
"product_enterprise_id" int4,
"product_enterprise" varchar(255) COLLATE "default",
"produce_first_areaid" int4,
"produce_second_areaid" int4,
"produce_third_areaid" int4,
"produce_first_addr" varchar(255) COLLATE "default",
"produce_second_addr" varchar(255) COLLATE "default",
"produce_third_addr" varchar(255) COLLATE "default",
"prod_detail_madeinid" int4,
"prod_detail_factory_addrid" int4,
"prod_detail_factory_name_addrid" int4,
"catid_y" int8,
"first_cat" varchar(255) COLLATE "default",
"second_cat" varchar(255) COLLATE "default",
"third_cat" varchar(255) COLLATE "default",
"forth_cat" varchar(255) COLLATE "default"
)
WITH (OIDS=FALSE)
;

ALTER TABLE "gather_taobao"."ir_taobao_product_trade_china_2019" OWNER TO "pengxiao";

COMMENT ON TABLE "gather_taobao"."ir_taobao_product_trade_china_2019" IS '2019年淘宝产品交易汇总表';

# 凡是涉及到字符类型比较或排序的地方，都会和COLLATE有关。COLLATE是一个算法语句，主要用于对字符进行排序，经常出现在表的创建语句中。

***********************************************************************************************************************************
# 表的结构 `erp_alert_sku`
--
CREATE TABLE `erp_alert_sku` (
  `id` int(10) UNSIGNED NOT NULL,
  `site_id` int(10) UNSIGNED NOT NULL DEFAULT '0' COMMENT '站点ID',
  `order_id` int(10) UNSIGNED NOT NULL DEFAULT '0' COMMENT '订单ID',
  `order_product_id` int(10) UNSIGNED NOT NULL DEFAULT '0' COMMENT '订单产品ID',
  `pl_spu` varchar(191) COLLATE utf8mb4_unicode_ci NOT NULL DEFAULT '' COMMENT '产品SPU码',
  `pl_sku` varchar(191) COLLATE utf8mb4_unicode_ci NOT NULL DEFAULT '' COMMENT '产品SKU码',
  `alert_type` enum('purchase','supply','storage','delivery') COLLATE utf8mb4_unicode_ci NOT NULL COMMENT '预警类型',
  `delayed_time` int(10) UNSIGNED NOT NULL DEFAULT '0' COMMENT '延误的时间（单位：秒）',
  `paid_at` timestamp NULL DEFAULT NULL COMMENT '订单支付时间',
  `placed_at` timestamp NULL DEFAULT NULL COMMENT '下单时间',
  `created_at` timestamp NULL DEFAULT NULL,
  `updated_at` timestamp NULL DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;


# 表的结构 `erp_site`
--
CREATE TABLE `erp_site` (
  `id` int(10) UNSIGNED NOT NULL,
  `erp_site_id` int(11) NOT NULL DEFAULT '0' COMMENT 'erp站点id',
  `name` varchar(100) COLLATE utf8mb4_unicode_ci NOT NULL DEFAULT '' COMMENT 'erp站点名称',
  `short_name` varchar(20) COLLATE utf8mb4_unicode_ci NOT NULL DEFAULT '' COMMENT 'erp站点简称',
  `created_at` timestamp NULL DEFAULT NULL,
  `updated_at` timestamp NULL DEFAULT NULL,
  `deleted_at` timestamp NULL DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

# 表的结构 `inventory_source_blogs`
--
CREATE TABLE `inventory_source_blogs` (
  `id` int(10) UNSIGNED NOT NULL,
  `stat_date` date DEFAULT NULL COMMENT '日期',
  `instore_total_num` int(11) NOT NULL DEFAULT '0' COMMENT '入库总数',
  `instore_total_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '入库总金额',
  `instore_web_num` int(11) NOT NULL DEFAULT '0' COMMENT '网采入库数',
  `instore_web_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '网采入库金额',
  `instore_store_num` int(11) NOT NULL DEFAULT '0' COMMENT '档口入库数',
  `instore_store_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '档口入库金额',
  `instore_factory_num` int(11) NOT NULL DEFAULT '0' COMMENT '工厂入库数',
  `instore_factory_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '工厂入库金额',
  `instore_other_num` int(11) NOT NULL DEFAULT '0' COMMENT '其它入库数',
  `instore_other_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '其它入库金额',
  `created_at` timestamp NULL DEFAULT NULL,
  `updated_at` timestamp NULL DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

#  表的结构 `inventory_use_blogs`
--
CREATE TABLE `inventory_use_blogs` (
  `id` int(10) UNSIGNED NOT NULL,
  `stat_date` date DEFAULT NULL COMMENT '日期',
  `instore_num` int(11) NOT NULL DEFAULT '0' COMMENT '入库数',
  `instore_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '入库金额',
  `outbound_num` int(11) NOT NULL DEFAULT '0' COMMENT '出库量',
  `outbound_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '出库金额',
  `inventory_num` int(11) NOT NULL DEFAULT '0' COMMENT '存货量',
  `inventory_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '存货金额',
  `created_at` timestamp NULL DEFAULT NULL,
  `updated_at` timestamp NULL DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

# -- 表的结构 `migrations`
CREATE TABLE `migrations` (
  `id` int(10) UNSIGNED NOT NULL,
  `migration` varchar(191) COLLATE utf8mb4_unicode_ci NOT NULL,
  `batch` int(11) NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

# 表的结构 `order_state`
CREATE TABLE `order_state` (
  `id` int(10) UNSIGNED NOT NULL,
  `date` date DEFAULT NULL COMMENT '日期',
  `site_id` int(10) UNSIGNED NOT NULL DEFAULT '0' COMMENT '站点ID',
  `site_name` varchar(100) COLLATE utf8mb4_unicode_ci NOT NULL DEFAULT '' COMMENT '站点名称',
  `product_price` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '产品价格',
  `order_freight` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '运费',
  `order_discount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '折扣',
  `order_exchange` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '换购差价',
  `income` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '总营收',
  `purchase_price` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购价',
  `purchase_freight` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购运费',
  `purchase_cancel_fee` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购取消',
  `purchase_return_price` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购退回',
  `purchase_return_freight` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购退回运费',
  `shipping_cost` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '发货费',
  `supply_fee` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '总供应费用',
  `purchase_estimate` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购预估',
  `shipping_estimate` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '发货预估',
  `order_refunds` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '退款额',
  `passage_fee` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '通道费',
  `order_qty` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '下单件数',
  `exchange_qty` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '换购件数',
  `purchase_qty` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购件数',
  `purchase_cancel_times` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购取消次数',
  `purchase_return_times` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '采购退回次数',
  `weight` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '发货重量',
  `shipping_qty` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '发货数量',
  `no_purchase_qty` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '未采购数量',
  `no_shipping_qty` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '未发货数量',
  `ad_fee` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '广告费',
  `profit_estimate` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '利润预估',
  `profit_rate` decimal(12,4) NOT NULL DEFAULT '0.0000' COMMENT '利润率',
  `roi` decimal(12,4) NOT NULL DEFAULT '0.0000' COMMENT 'ROI',
  `created_at` timestamp NULL DEFAULT NULL,
  `updated_at` timestamp NULL DEFAULT NULL,
  `s_pur_qty` int(10) UNSIGNED NOT NULL DEFAULT '0' COMMENT '应该采购的数量',
  `s_ship_qty` int(10) UNSIGNED NOT NULL DEFAULT '0' COMMENT '还没到达备货的数量'
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

# 表的结构 `outstock_blogs`
CREATE TABLE `outstock_blogs` (
  `id` int(10) UNSIGNED NOT NULL,
  `stat_date` date DEFAULT NULL COMMENT '日期',
  `web_site` int(11) NOT NULL COMMENT '站点',
  `order_outstock_product_num` int(11) NOT NULL DEFAULT '0' COMMENT '缺货件数',
  `order_product_num` int(11) NOT NULL DEFAULT '0' COMMENT '付款件数',
  `order_out_pur_product_rate` decimal(6,4) NOT NULL DEFAULT '0.0000' COMMENT '缺货件数占比采购总件数',
  `purchase_outstock_product_num` int(11) NOT NULL DEFAULT '0' COMMENT '缺货件数',
  `purchase_product_num` int(11) NOT NULL DEFAULT '0' COMMENT '采购总件数',
  `purchase_out_pur_product_rate` decimal(6,4) NOT NULL DEFAULT '0.0000' COMMENT '缺货件数占比采购总件数',
  `created_at` timestamp NULL DEFAULT NULL,
  `updated_at` timestamp NULL DEFAULT NULL,
  `order_product_purchase_num` int(11) NOT NULL DEFAULT '0' COMMENT '采购件数',
  `order_product_arrival_num` int(11) NOT NULL DEFAULT '0' COMMENT '采到件数',
  `purchase_product_twice_num` int(11) NOT NULL DEFAULT '0' COMMENT '二次采购件数',
  `order_unprocessed_product_num` int(11) NOT NULL DEFAULT '0' COMMENT '已付款未处理数量'
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

# 表的结构 `password_resets`
CREATE TABLE `password_resets` (
  `email` varchar(191) COLLATE utf8mb4_unicode_ci NOT NULL,
  `token` varchar(191) COLLATE utf8mb4_unicode_ci NOT NULL,
  `created_at` timestamp NULL DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

# -- 表的结构 `refundment_blogs`
CREATE TABLE `refundment_blogs` (
  `id` int(10) UNSIGNED NOT NULL,
  `stat_date` date DEFAULT NULL COMMENT '日期',
  `web_site` int(11) NOT NULL COMMENT '站点',
  `refund_order_num` int(11) NOT NULL DEFAULT '0' COMMENT '退款订单数',
  `pay_order_num` int(11) NOT NULL DEFAULT '0' COMMENT '付款订单数',
  `refund_pay_rate` decimal(6,4) NOT NULL DEFAULT '0.0000' COMMENT '退款订单数占比付款订单数',
  `refund_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '退款总金额',
  `total_amount` decimal(12,2) NOT NULL DEFAULT '0.00' COMMENT '订单总金额',
  `refund_amount_rate` decimal(6,4) NOT NULL DEFAULT '0.0000' COMMENT '退款总金额占比订单总金额',
  `refund_overall_order_num` int(11) NOT NULL DEFAULT '0' COMMENT '全单退款数',
  `refuse_true_order_num` int(11) NOT NULL DEFAULT '0' COMMENT '拒付单数',
  `refuse_true_rate` decimal(8,2) NOT NULL DEFAULT '0.00' COMMENT '拒付占比',
  `refuse_false_order_num` int(11) NOT NULL DEFAULT '0' COMMENT '非拒付单数',
  `refuse_false_rate` decimal(8,2) NOT NULL DEFAULT '0.00' COMMENT '非拒付占比',
  `refund_product_num` int(11) NOT NULL DEFAULT '0' COMMENT '退款总件数',
  `created_at` timestamp NULL DEFAULT NULL,
  `updated_at` timestamp NULL DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

# 表的结构 `users`
CREATE TABLE `users` (
  `id` int(10) UNSIGNED NOT NULL,
  `name` varchar(191) COLLATE utf8mb4_unicode_ci NOT NULL,
  `email` varchar(191) COLLATE utf8mb4_unicode_ci NOT NULL,
  `password` varchar(191) COLLATE utf8mb4_unicode_ci NOT NULL,
  `remember_token` varchar(100) COLLATE utf8mb4_unicode_ci DEFAULT NULL,
  `created_at` timestamp NULL DEFAULT NULL,
  `updated_at` timestamp NULL DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSE
